In [11]:
import pandas as pd
import tabula

initial_df = pd.read_excel("D:\\Projects\\Akros\\joseacevedogomez\\Notas 2024 media y clei\\EC-F04_CONSOLIDADO_10A.xlsx")
# Extract tables from the PDF
dfs = tabula.read_pdf("D:\\Projects\\Akros\\joseacevedogomez\\tecnicas\\CONSOLIDADO DE NOTAS 2024.pdf", pages='all')
df = pd.concat(dfs)
df

AttributeError: module 'tabula' has no attribute 'read_pdf'

In [9]:
def process_workbook(route, quarter):
    df = pd.read_excel(route, sheet_name=quarter ) 
    
    df.columns = df.iloc[8].tolist()
    df = df.iloc[9:,2:]
    df = df.loc[:, 'APELLIDOS Y NOMBRES':'Total pérdidas'].iloc[:, :-1]
    df = df.set_index('APELLIDOS Y NOMBRES')

    df = df.infer_objects()
    replacement_dict = {
        'DBAJO': 'BJ',
        'DB': 'B',
        'DA': 'A',
        'DS': 'S'
        
    }
    df.replace(replacement_dict, inplace=True)
    
    # Replace all other values with None
    for column in df.columns:
        df[column] = df[column].map(lambda x: None if x not in replacement_dict.values() else x)
    
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
    
    
    return df

def extract_data(route, quarter):
    df = pd.read_excel(route, sheet_name=quarter ) 
    
    data = {
        'Grado':None,
        'Grupo':None,
        'Periodo': None
    }
    grado_grupo = df.iloc[4,1]
    
    if len(grado_grupo.split(': ')) > 1:
        grado_grupo = grado_grupo.split(': ')[1]
    
    if len(df.iloc[5,1].split(': ')) > 1:
        data['Periodo'] = df.iloc[5,1].split(': ')[1].strip()
    else:
        data['Periodo'] = df.iloc[5,1]
    if len(grado_grupo.split()) > 1:
        data['Grado'] = grado_grupo.split()[0]
        data['Grupo'] = grado_grupo.split()[1]
    else:
        data['Grado'] = grado_grupo.strip()
    return  data



In [4]:
import unicodedata
import re


def remove_tildes(text):
    """Removes the accents of each vowel in a text
    
    :param str text: the text with vowels with accents 
    :return str: the text without the accents 
    """
    # Normalize and remove accents
    text = ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
    )
    return text.strip()

def clean_name(name):
    """Cleans a name by removing numbers and accents.
    
    :param str name: The name to be cleaned.
    :return str: The cleaned name.
    """
    # Remove numbers using regex
    name = re.sub(r'\d+', '', name)
    
    # Remove tildes (accents) from vowels
    name = remove_tildes(name)
    
    # Return the cleaned name
    return name.strip()

def clean_column_name(column):
    """Cleans the column name by:
    - Replacing spaces with underscores.
    - Removing accents from vowels.
    
    :param str column: The original column name.
        
    :return str: The cleaned column name.
    """
    # Replace spaces with underscores
    column = column.replace(" ", "_")
    
    # Normalize and remove accents
    column = remove_tildes(column)
    
    # Convert to lowercase and strip any extra whitespace
    return column.strip().lower()

In [8]:
example_route = ("D:\\Projects\\Akros\\joseacevedogomez\\Notas 2024 media y clei\\EC-F04_CONSOLIDADO_10A.xlsx")

def process_df(route):
    output = []
    df = process_workbook(route, 2)
    data = extract_data(route, 2)   
    for index, row in df.iterrows():
        for column in df.columns:
            
            estudiante = clean_name(index)
            
            materia = clean_column_name(column)
            
            nota = row[column]
            
            grado = clean_column_name(data['Grado'])
            
            grupo = data['Grupo']
            
            output.append((estudiante, materia, nota, grado, grupo))
    return output
            
process_df(example_route)

KeyError: 'APELLIDOS Y NOMBRES'

In [32]:
import os


def process_folder(folder_path):
    """
    Process all SQL files in a folder and save their processed data as JSON.
    """

    if not os.path.exists(folder_path):
        print(f"Folder does not exist: {folder_path}")
        return

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        if os.path.isfile(file_path) and file_name.lower().endswith('.xlsx'):
            print(f"\nProcessing file: {file_path}")
            # El periodo 3 contiene las notas definitivas
            print(len(process_df(file_path)))

process_folder("D:\\Projects\\Akros\\joseacevedogomez\\Notas 2024")
            



Processing file: D:\Projects\Akros\joseacevedogomez\Notas 2024\EC-F04_CONSOLIDADO_1A_COLINA.xlsx
280

Processing file: D:\Projects\Akros\joseacevedogomez\Notas 2024\EC-F04_CONSOLIDADO_1A_COSTA_RICA.xlsx
270

Processing file: D:\Projects\Akros\joseacevedogomez\Notas 2024\EC-F04_CONSOLIDADO_1B_COSTA_RICA.xlsx


AttributeError: 'float' object has no attribute 'split'

In [33]:


def process_workbook(route, quarter):
    df = pd.read_excel(route, sheet_name=quarter)

    df.columns = [str(col).strip() for col in df.iloc[8].tolist()]
    df = df.iloc[9:, 2:]
    df = df.loc[:, :'TP'].iloc[:, :-1]
    df = df.set_index('APELLIDOS Y NOMBRES')

    df = df.infer_objects()
    original_df = df
    # df = replace_notes(df)
    return df

process_workbook(example_route, 2)

,QUÍ,BIO,FIS,SOC,CEP,ART,ETI,EF,REL,LC,ING,MAT,TEC,FIL,MT
APELLIDOS Y NOMBRES,,,,,,,,,,,,,,,
Alvarez Ocampo Juan Esteban,DS,DS,DA,DS,DS,DA,DS,DA,DS,DB,DS,DS,DA,DS,DB
Arango Graciano Jimena,DA,DA,DB,DS,DS,DS,DS,DA,DS,DB,DA,DS,DS,DS,DB
Arboleda Vahos Yulithza,DB,DA,DB,DS,DS,DA,DS,DA,DS,DS,DA,DA,DS,DS,DB
Becerra Caicedo David,DA,DA,DB,DS,DS,DS,DA,DA,DA,DA,DS,DA,DA,DA,DB
Castaño Lujan Samuel Andrey,DB,DA,DA,DS,DS,DA,DA,DA,DA,DA,DA,DA,DA,DA,DB
Chavarria Jaramillo Juan Sebastian,DA,DA,DB,DS,DS,DA,DA,DB,DA,DB,DB,DS,DB,DA,DB
Chavez Perez Helen Johana,DB,DA,DA,DA,DA,DA,DS,DA,DS,DB,DB,DA,DA,DS,DB
Correa Causado Jesus David SE RETIRO EL 27 DE FEB/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cuevas Bello Sheyra Camila Nueva 2024,DB,DB,DA,DS,DS,DA,DA,DB,DA,DA,DA,DA,DA,DA,DB
